In [50]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from utils import *
import numpy as np
from sklearn.cross_validation import train_test_split
from tflearn.data_utils import to_categorical, pad_sequences
from experiments import *
from sklearn.metrics import accuracy_score
import tflearn
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from models_dict import *

In [75]:
x, y, mask = load_data_will_student_solve_next_problem(hoc_num=18, minlen=2, y_is_seq=True, with_mask=True)

Loading data...
Data loaded.


In [76]:
mask = np.broadcast_to(mask, (mask.shape[0], mask.shape[1], y.shape[2]))
print mask[0]

[[ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [77]:
x_train, x_test, y_train, y_test, mask_train, mask_test = train_test_split(x, y, mask, test_size=0.1, random_state=42)

In [72]:
print x_train.shape
print mask_train.shape

(85464, 10, 50)
(85464, 10, 2)


In [67]:
model_id = "predict_next_prob_binary"
n_timesteps = x_train.shape[1]
n_hidden = 128
n_inputdim = x_train.shape[2]
n_classes = 2
architecture = 'lstm_predict_binary'
check_model_exists_or_create_new(model_id, n_timesteps, n_inputdim, n_hidden, n_classes, architecture)

A model with the same model_id 'predict_next_prob_binary' already exists. 
No differences found. Yay! 


In [68]:
tensorboard_dir = '../tensorboard_logs/' + model_id + '/'
checkpoint_path = '../checkpoints/' + model_id + '/'
print (tensorboard_dir)
print (checkpoint_path)

check_if_path_exists_or_create(tensorboard_dir)
check_if_path_exists_or_create(checkpoint_path)
    
tensorboard_verbose = 2 
max_checkpoints = 3
clip_gradients = 5.0

../tensorboard_logs/predict_next_prob_binary/
../checkpoints/predict_next_prob_binary/


In [78]:
print (y * mask)[10]

[[ 0.  1.]
 [ 0.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [ ]:
def sequential_categorical_crossentropy(y_pred, y_true, mask=None):
    """ Categorical Crossentropy.

    Computes cross entropy between y_pred (logits) and y_true (labels).
    Arguments:
        y_pred: `3D-Tensor`, Predicted values. shape (None, n_timesteps, )
        y_true: `3D-Tensor`, Targets (labels), a probability distribution.
        mask: `3D-Tensor`, masks out the padded timesteps. (since sequences are not the equal length)
    """
    with tf.name_scope("Sequential_Crossentropy"):
        n_timesteps = y_pred.shape
        y_pred = tf.reshape(y_pred, [-1, n_timesteps * n_classes], name='Reshape')
        
        
        y_pred /= tf.reduce_sum(y_pred,
                                reduction_indices=len(y_pred.get_shape())-1,
                                keep_dims=True)
        # manual computation of crossentropy
        y_pred = tf.clip_by_value(y_pred, tf.cast(_EPSILON, dtype=_FLOATX),
                                  tf.cast(1.-_EPSILON, dtype=_FLOATX))
        if mask:
            seq_cross_entropy = - tf.reduce_sum(tf.mul(y_true, mask) * tf.log(y_pred),
                                                reduction_indices=len(y_pred.get_shape()) - 1)
        else:
            seq_cross_entropy = - tf.reduce_sum(y_true * tf.log(y_pred),
                                   reduction_indices=len(y_pred.get_shape())-1)
        return tf.reduce_mean(seq_cross_entropy)

In [74]:
with tf.Graph().as_default():
    X = tf.placeholder(shape=(None, n_timesteps, n_inputdim), dtype=tf.float32, name="X")
    mask = tf.placeholder(shape=(None, n_timesteps, n_classes), dtype=tf.float32, name="mask")
    Y = tf.placeholder(shape=(None, n_timesteps, n_classes), dtype=tf.float32, name="Y")

    tf.add_to_collection(tf.GraphKeys.INPUTS, X)
    tf.add_to_collection(tf.GraphKeys.LAYER_TENSOR + '/' + "X", X)

    tf.add_to_collection(tf.GraphKeys.INPUTS, mask)
    tf.add_to_collection(tf.GraphKeys.LAYER_TENSOR + '/' + "mask", mask)

    if Y not in tf.get_collection(tf.GraphKeys.TARGETS):
        tf.add_to_collection(tf.GraphKeys.TARGETS, Y)

    net = tflearn.lstm(X, n_hidden, return_seq=True)
    net = tflearn.lstm(net, n_classes, return_seq=True)
    # outgoing tensor of shape [None, max_seq_len, 2]
#     net = tf.reshape(net, [-1, n_timesteps * n_classes], name='Reshape')
    # mask y_pred to exclude certain timesteps
    
    masked_y = tf.mul(Y, mask) # element-wise multiplication with broadcasting
    
    net = tf.reshape(net, [-1, n_timesteps * n_classes], name='reshape_pred')
    masked_y = tf.reshape(masked_y, [-1, n_timesteps * n_classes], name='reshape_true_y')
    
    loss = tflearn.categorical_crossentropy(net, masked_y)

#     loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(net, masked_y))
#     loss = sequential_categorical_crossentropy(net, Y, mask=mask)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    
    # TODO: Reformulate the accuracy function
    accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(net, 1), tf.argmax(masked_y, 1)), tf.float32),
        name='acc')

    # Using TFLearn Trainer
    # Define a training op (op for backprop, only need 1 in this model)
    trainop = tflearn.TrainOp(loss=loss, optimizer=optimizer,
                              metric=accuracy, batch_size=128)
    
    tf.add_to_collection(tf.GraphKeys.TRAIN_OPS, trainop)
    

    # Create Trainer, providing all training ops. Tensorboard logs stored
    # in /tmp/tflearn_logs/. It is possible to change verbose level for more
    # details logs about gradients, variables etc...
    trainer = tflearn.Trainer(train_ops=trainop, 
                               clip_gradients=clip_gradients,
                               tensorboard_dir=tensorboard_dir,
                               tensorboard_verbose=tensorboard_verbose,
                               checkpoint_path=checkpoint_path,
                               max_checkpoints=max_checkpoints)

    trainer.fit({X: x_train, mask: mask_train, Y: y_train}, 
                val_feed_dicts={X: x_test, mask: mask_test, Y: y_test},
                n_epoch=10, show_metric=True)
    

Training Step: 6680  | total loss: 14.01141
| Optimizer | epoch: 010 | loss: 14.01141 - acc: 0.0216 | val_loss: 13.58568 - val_acc: 0.0180 -- iter: 85464/85464
Training Step: 6680  | total loss: 14.01141
| Optimizer | epoch: 010 | loss: 14.01141 - acc: 0.0216 | val_loss: 13.58568 - val_acc: 0.0180 -- iter: 85464/85464
--


Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [79]:
with tf.Graph().as_default():
    X = tf.placeholder(shape=(None, n_timesteps, n_inputdim), dtype=tf.float32, name="X")
    mask = tf.placeholder(shape=(None, n_timesteps, n_classes), dtype=tf.float32, name="mask")
    Y = tf.placeholder(shape=(None, n_timesteps, n_classes), dtype=tf.float32, name="Y")

    tf.add_to_collection(tf.GraphKeys.INPUTS, X)
    tf.add_to_collection(tf.GraphKeys.LAYER_TENSOR + '/' + "X", X)

    tf.add_to_collection(tf.GraphKeys.INPUTS, mask)
    tf.add_to_collection(tf.GraphKeys.LAYER_TENSOR + '/' + "mask", mask)

    if Y not in tf.get_collection(tf.GraphKeys.TARGETS):
        tf.add_to_collection(tf.GraphKeys.TARGETS, Y)

    net = tflearn.lstm(X, n_hidden, return_seq=True)
    net = tflearn.lstm(net, n_classes, return_seq=True)
    # outgoing tensor of shape [None, max_seq_len, 2]
#     net = tf.reshape(net, [-1, n_timesteps * n_classes], name='Reshape')
    # mask y_pred to exclude certain timesteps
    
    masked_y = tf.mul(Y, mask) # element-wise multiplication with broadcasting
    
    net = tf.reshape(net, [-1, n_timesteps * n_classes], name='reshape_pred')
    masked_y = tf.reshape(masked_y, [-1, n_timesteps * n_classes], name='reshape_true_y')
    
    loss = tflearn.categorical_crossentropy(net, masked_y)

#     loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(net, masked_y))
#     loss = sequential_categorical_crossentropy(net, Y, mask=mask)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    
    # TODO: Reformulate the accuracy function
    accuracy = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(net, 1), tf.argmax(masked_y, 1)), tf.float32),
        name='acc')

    # Using TFLearn Trainer
    # Define a training op (op for backprop, only need 1 in this model)
    trainop = tflearn.TrainOp(loss=loss, optimizer=optimizer,
                              metric=accuracy, batch_size=128)
    
    tf.add_to_collection(tf.GraphKeys.TRAIN_OPS, trainop)
    
    
    model = tflearn.DNN(net, tensorboard_verbose=2, tensorboard_dir=tensorboard_dir, \
                            checkpoint_path=checkpoint_path, max_checkpoints=3)
    

    date_time_string = datetime.datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
    run_id = "{}_{}".format(model_id, date_time_string)
    model.fit({'X': x_train, 'mask': mask_train}, y_train,
                validation_set=({'X': x_test, 'mask': mask_test}, y_test),
                n_epoch=32, show_metric=True, snapshot_step=100, run_id=run_id)

Exception: Feed dict asks for variable named 'X' but no such variable is known to exist